Trabalho Prático 1 Aprendizado de máquina: Redes neurais de duas camadas
Aluno: Nélio Cezar Muniz Sampaio
Matrícula: 2015042150

In [27]:
# Importando bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
# Importando dados
dataset = pd.read_csv('data_tp1')

In [29]:
#Separando em (x, y)
x = dataset.iloc[:, 1:785].values
y = dataset.iloc[:, 0]
#print( x[0] )
#print(y)

In [30]:
print("y type: ", type(y))

y type:  <class 'pandas.core.series.Series'>


In [31]:
# Transformando saída em vetor.
y2 = y
conv_arr = y2.values

#print(y)
#print(conv_arr)

d = [ [] for i in range(len(conv_arr))]
for ind, i in enumerate(conv_arr):
    #print("i:", i)
    for j in range(10):
        #print("j:", j)
        if i==j:
            d[ind].append(1)
        else:
            d[ind].append(0)

df = pd.DataFrame(data=d)
#df
            
#for ind, i in enumerate(d):
#    print(conv_arr[ind])
#    print(i)

In [40]:
#Separando em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x, df, test_size = 0.2, random_state = 0 )

#print(x_train)
#print(y_train)
#print(x_test)
#print(y_test)

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense


In [42]:
# Inicializando a ANN
classifier = Sequential()

# Adicionando input layer e a hidden layer
# units: neuronios na hidden layer; kernel_initializer: inicializar os pesos aleatoriamente; 
# activation: função de ativação; input_dim: neurônios na input layer.
classifier.add( Dense(units=50, kernel_initializer='uniform', activation='relu', input_dim=784) )


In [43]:
# Adicionando a output Layer
classifier.add( Dense(units=10, kernel_initializer='uniform', activation='sigmoid') )

In [45]:
# Compilando
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
classifier.fit(x_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
3999/3999 [==============================] - 1s 204us/step - loss: 0.1865 - acc: 0.9467
Epoch 2/100
3999/3999 [==============================] - 0s 99us/step - loss: 0.0713 - acc: 0.9754
Epoch 3/100
3999/3999 [==============================] - 0s 97us/step - loss: 0.0537 - acc: 0.9808
Epoch 4/100
3999/3999 [==============================] - 0s 96us/step - loss: 0.0428 - acc: 0.9850
Epoch 5/100
3999/3999 [==============================] - 0s 97us/step - loss: 0.0424 - acc: 0.9859
Epoch 6/100
3999/3999 [==============================] - 0s 96us/step - loss: 0.0355 - acc: 0.9879
Epoch 7/100
3999/3999 [==============================] - 0s 99us/step - loss: 0.0373 - acc: 0.9878
Epoch 8/100
3999/3999 [==============================] - 0s 100us/step - loss: 0.0343 - acc: 0.9888
Epoch 9/100
3999/3999 [==============================] - 0s 99us/step - loss: 0.0340 - acc: 0.9885
Epoch 10/100
3999/3999 [==============================] - 0s 99us/step - loss: 0.0253 - acc: 0.9917
Epoch 1

In [69]:
y_pred = classifier.predict(x_test)
#print (y_pred)

r = []

for i in y_pred:
    r.append( i.argmax() )

#for i in y_test:
#    print(i)
#print(y_test.get_values())    
    
cont_certos=0
cont_errados=0
for index, i in enumerate(r):
    previsto=i
    resposta=y_test.get_values()[index].argmax()
    
    if previsto==resposta:
        cont_certos += 1
    else:
        cont_errados += 1
    
    print("r:", i)
    print("ans: ", y_test.get_values()[index].argmax())

#y_pred = (y_pred > 0.5)

r: 3
ans:  3
r: 9
ans:  9
r: 8
ans:  8
r: 1
ans:  1
r: 5
ans:  5
r: 4
ans:  4
r: 5
ans:  5
r: 3
ans:  3
r: 1
ans:  1
r: 6
ans:  6
r: 4
ans:  4
r: 2
ans:  2
r: 8
ans:  9
r: 9
ans:  9
r: 0
ans:  0
r: 5
ans:  5
r: 4
ans:  4
r: 9
ans:  9
r: 3
ans:  3
r: 9
ans:  9
r: 1
ans:  1
r: 4
ans:  4
r: 0
ans:  0
r: 4
ans:  4
r: 5
ans:  5
r: 1
ans:  1
r: 0
ans:  0
r: 3
ans:  3
r: 7
ans:  7
r: 7
ans:  7
r: 0
ans:  0
r: 8
ans:  8
r: 6
ans:  6
r: 4
ans:  4
r: 3
ans:  3
r: 7
ans:  7
r: 9
ans:  9
r: 6
ans:  6
r: 1
ans:  1
r: 6
ans:  6
r: 6
ans:  6
r: 3
ans:  3
r: 0
ans:  0
r: 4
ans:  4
r: 7
ans:  7
r: 0
ans:  0
r: 4
ans:  4
r: 3
ans:  3
r: 9
ans:  9
r: 3
ans:  3
r: 4
ans:  4
r: 7
ans:  7
r: 7
ans:  7
r: 4
ans:  2
r: 1
ans:  1
r: 1
ans:  7
r: 2
ans:  2
r: 7
ans:  7
r: 0
ans:  0
r: 2
ans:  2
r: 0
ans:  0
r: 2
ans:  2
r: 6
ans:  6
r: 8
ans:  8
r: 2
ans:  2
r: 7
ans:  2
r: 4
ans:  9
r: 8
ans:  8
r: 7
ans:  7
r: 8
ans:  5
r: 9
ans:  9
r: 9
ans:  7
r: 0
ans:  0
r: 0
ans:  0
r: 6
ans:  6
r: 8
ans:  8
r: 1
ans:  1

r: 0
ans:  0
r: 0
ans:  0
r: 0
ans:  0
r: 5
ans:  5
r: 4
ans:  4
r: 3
ans:  3
r: 3
ans:  3
r: 1
ans:  1
r: 6
ans:  6
r: 5
ans:  5
r: 2
ans:  2
r: 6
ans:  6
r: 3
ans:  3
r: 6
ans:  6
r: 5
ans:  5
r: 2
ans:  2
r: 7
ans:  7
r: 9
ans:  9
r: 4
ans:  4
r: 7
ans:  7
r: 5
ans:  5
r: 0
ans:  0
r: 6
ans:  6
r: 4
ans:  4
r: 9
ans:  9
r: 3
ans:  3
r: 2
ans:  2
r: 7
ans:  7
r: 7
ans:  7
r: 2
ans:  2
r: 0
ans:  0
r: 9
ans:  9
r: 4
ans:  2
r: 5
ans:  5
r: 2
ans:  2
r: 0
ans:  0
r: 9
ans:  9
r: 3
ans:  3
r: 1
ans:  1
r: 2
ans:  2
r: 6
ans:  6
r: 9
ans:  9
r: 7
ans:  7
r: 4
ans:  9
r: 0
ans:  0
r: 5
ans:  5
r: 2
ans:  2
r: 5
ans:  5
r: 0
ans:  0
r: 6
ans:  6
r: 4
ans:  4
r: 7
ans:  7
r: 3
ans:  3
r: 2
ans:  2
r: 6
ans:  6
r: 5
ans:  5
r: 1
ans:  1
r: 0
ans:  0
r: 5
ans:  5
r: 8
ans:  8
r: 5
ans:  5
r: 9
ans:  9
r: 4
ans:  4
r: 0
ans:  0
r: 9
ans:  9
r: 8
ans:  8
r: 5
ans:  5
r: 2
ans:  2
r: 3
ans:  3
r: 9
ans:  9
r: 8
ans:  8
r: 4
ans:  7
r: 8
ans:  3
r: 8
ans:  8
r: 8
ans:  0
r: 0
ans:  0
r: 2
ans:  2

In [76]:
print("corretos: ", cont_certos)
print("errados: ", cont_errados)

corretos:  923
errados:  77


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y)

In [7]:
# Definição da função sigmoida
def sigmoid( x, derivada=False ):
    if derivada==True:
        return x*(1-x)
    return 1/(1+np.exp(-x))